In [2]:
%reset
%matplotlib inline
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean
import warnings; warnings.filterwarnings('ignore')
print('ready')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
ready


In [3]:
grid_data=xr.open_zarr("/scratch/ag7531/shareElizabeth/grid_dataforeli")
uv_data  =xr.open_zarr("/scratch/ag7531/shareElizabeth/uv_dataforeli")

In [4]:
#Uncomment to print the uv_data information:
#uv_data 

In [5]:
#Uncomment to print the grid_data information:
#grid_data

In [6]:
##Reading in surface velocity data and the (xu_ocean, yu_ocean) velocity points:
usurf = uv_data.usurf.values[-1,:,:] #Reading in the last time index
vsurf = uv_data.vsurf.values[-1,:,:] #Reading in the last time index
yu_ocean = grid_data.yu_ocean.values 

xu_ocean = grid_data.xu_ocean.values

##To show that the gridfile and datafile have the same yu and xu definitions:
#yu_ocean_datafile = uv_data.yu_ocean.values
#xu_ocean_datafile = uv_data.xu_ocean.values
#print(np.nanmax(yu_ocean_datafile-yu_ocean)); print(np.nanmax(xu_ocean_datafile-xu_ocean))

##Reading in latitude and longitude at velocity points:
geolat = grid_data.geolat_c.values
geolon = grid_data.geolon_c.values
##Reading in the grid spacing for thickness points:
dyt = grid_data.dyt.values
dxt = grid_data.dxt.values
print(usurf.shape)
print('finished')

(2700, 3600)
finished


In [12]:
#Preparing the inputs
print(usurf.shape)
print(dxt.shape)
print(dyt.shape)
print(mix.shape)

(2700, 3600)
(2700, 3600)
(2700, 3600)
(2700, 3600)


In [7]:
def MOM5_Laplacian(a, dxt, dyt):
    #a is the field for which Laplacian is being computed
    #dxt is x-direction grid spacing for a t-cell
    #dyt is y-direction grid spacing for a t-cell

    #Initialize empty arrays for fx, fy (first derivatives), and LAP_U (final Laplacian result)
    fx = np.empty(a.shape)
    fy = np.empty(a.shape)
    LAP_U = np.empty(a.shape)
    
    for i in range(1,a.shape[0]-1):
        for j in range(a.shape[1]):
            fx[i,j]=(a[i+1,j]-a[i-1,j])/(dxt[i,j]+dxt[i+1,j])              
    print('Computed fx')
    for i in range(a.shape[0]):
        for j in range(1,a.shape[1]-1):
            fy[i,j]=(a[i,j+1]-a[i,j-1])/(dyt[i,j]+dyt[i,j+1]) 
    print('Computed fy') 
    for i in range(1,a.shape[0]-1):
        for j in range(1,a.shape[1]-1):
            LAP_U[i,j]=(fx[i+1,j]-fx[i-1,j])/(dxt[i,j]+dxt[i+1,j])+(fy[i,j+1]-fy[i,j-1])/(dyt[i,j]+dyt[i,j+1])
    print('Computed LAP_U')

    return LAP_U


In [9]:
u_laplacian=MOM5_Laplacian(usurf,dxt,dyt)
v_laplacian=MOM5_Laplacian(vsurf,dxt,dyt)

Computed fx
Computed fy
Computed LAP_U
Computed fx
Computed fy
Computed LAP_U


In [10]:
%pwd

'/home/eay8226/gcm-filters/notebooks'

In [24]:
%cd /home/eay8226/gcm-filters/gcm_filters
%ls
import kernels.py
#%cd /home/eay8226/gcm-filters/notebooks

/home/eay8226/gcm-filters/gcm_filters
__init__.py  __pycache__/  filter.py  gpu_compat.py  kernels.py


ImportError: attempted relative import with no known parent package